In [13]:
%load_ext autoreload
%autoreload 2

import sys
import pathlib

import numpy as np
import pandas as pd
import xarray as xr
import copernicusmarine
from loguru import logger
from tqdm.dask import TqdmCallback as ProgressBar

import oceansoda_ethzv2 as ocn
import oceansoda_ethzv2.data.utils.processors as p
import oceansoda_ethzv2.data.utils.checker as checker
from oceansoda_ethzv2.data.soda import SODADataset
from oceansoda_ethzv2.data import occci
from oceansoda_ethzv2.data.utils import config

logger.remove()
logger.add(sys.stdout, level='TRACE')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2

GroupNotFoundError: group not found at path ''

<xarray.Dataset> Size: 404kB
Dimensions:               (time: 46, lat: 720)
Coordinates:
  * time                  (time) datetime64[ns] 368B 2024-01-05 ... 2024-12-29
  * lat                   (lat) float64 6kB -89.88 -89.62 -89.38 ... 89.62 89.88
Data variables:
    xco2atm_mauna_loa     (time) float32 184B 143.1 142.6 142.3 ... 146.0 146.2
    xco2mbl_noaa          (time, lat) float32 132kB 417.5 417.5 ... 431.3 431.3
    xco2mbl_noaa_regress  (time, lat) float32 132kB 417.5 417.5 ... 431.3 431.3
    xco2mbl_noaa_raw      (time, lat) float32 132kB nan nan nan ... nan nan nan
Attributes:
    source:   https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_weekly_mlo.csv
    units:    ppm

In [2]:
validator = ocn.data.utils.ZarrYearValidator(error_handling='warn')

In [3]:
catalog = config.read_dataset_config('./dataset_sources.yaml')

In [13]:
soda = SODADataset(**catalog['soda']['soda_3_15_2'])

In [21]:
pathlib.Path("/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_19.nc").exists()
pathlib.Path("/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1984_10_04.nc").exists()

False

In [22]:
soda.get_timestep_unprocessed(time=pd.Timestamp('1981-10-14'))

2025-05-30 11:35:51.374 | WARNING  | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:413 - File ../data/8D_025/mld_soda-8D_025.zarr/1981 does not exist, but the timestep [year=1981, index=35] is not the first step in the year
2025-05-30 11:35:51.375 | DEBUG    | oceansoda_ethzv2.data.soda:_get_unprocessed_timestep_local:4 - Getting local SODA data at: /Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_{time:%Y_%m_%d}.nc
2025-05-30 11:35:51.377 | TRACE    | oceansoda_ethzv2.data.soda:_get_unprocessed_timestep_local:8 - Created paths for SODA that still need to be checked: 
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_03.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_04.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_05.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_06.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_07.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981_10_08.nc
/Users/luke/Downloads/soda3.15.2_5dy_ocean_reg_1981

<xarray.Dataset> Size: 11MB
Dimensions:   (time: 4, lat: 330, lon: 720)
Coordinates:
  * lat       (lat) float64 3kB -74.75 -74.25 -73.75 ... 88.75 89.25 89.75
  * time      (time) datetime64[ns] 32B 1981-10-04T13:00:00 ... 1981-10-19T13...
  * lon       (lon) float64 6kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
Data variables:
    sss_soda  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 330, 720), meta=np.ndarray>
    ssh_soda  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 330, 720), meta=np.ndarray>
    mld_soda  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 330, 720), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 1.9.7.1 (http:/...
    Conventions:               CF-1.6
    history:                   Tue Aug 31 14:43:03 2021: cdo -f nc -b F32 cop...
    filename:                  ./mom5_regrided.nc                   
    title:                     MOM5_SODA_3.15.2
    grid_type:                 mosaic
    grid_tile:                 1
    CDO:                       Climate Data Operators version 1.9.7.1 (http:/...
    processing_renamed_vars:   {'xt_ocean': 'lon', 'yt_ocean': 'lat', 'st_oce...
    processing_squeezed_dims:  []
    processing_lon360_to_180:  True
    requested_time:            1981-10-12 00:00:00

In [7]:
oisst = ocn.data.HTTPDailyDataset(**catalog['ceda']['sst_noaa'])

In [8]:
oisst.write_timestep_to_disk(2025, 0)


2025-05-28 12:37:41.903 | WARNING  | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:368 - Given timestep [year=2025, index=0] already exists in ../data/8D_025/ice_noaa-8D_025.zarr/2025


In [9]:
a = ocn.data.HTTPDailyDataset(**catalog['ceda']['sstcci'])
b = ocn.data.HTTPDailyDataset(**catalog['ceda']['ssscci'])


In [10]:
ds = b.get_timestep(2020, 0)

2025-05-28 12:37:43.701 | WARNING  | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:368 - Given timestep [year=2020, index=0] already exists in ../data/8D_025/sss_cci-8D_025.zarr/2020
2025-05-28 12:37:43.713 | TRACE    | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:365 - Given timestep [year=2020, index=0] already exists in ../data/8D_025/sss_cci-8D_025.zarr/2020
2025-05-28 12:37:43.714 | TRACE    | oceansoda_ethzv2.data.ceda:get_ceda_version_from_time:143 - No versions provided. Returning an empty string.
2025-05-28 12:37:43.715 | TRACE    | oceansoda_ethzv2.data.ceda:get_ceda_version_from_time:143 - No versions provided. Returning an empty string.
2025-05-28 12:37:43.715 | TRACE    | oceansoda_ethzv2.data.ceda:get_ceda_version_from_time:143 - No versions provided. Returning an empty string.
2025-05-28 12:37:43.715 | TRACE    | oceansoda_ethzv2.data.ceda:get_ceda_version_from_time:143 - No versions provided. Returning an empty string.
2025-05-28 12:37:43.715 | TRAC

In [57]:
b.write_timestep_to_disk(2020, 0)

2025-05-28 12:29:45.729 | WARNING  | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:368 - Given timestep [year=2020, index=0] already exists in ../data/8D_025/sss_cci_sigma_uncert-8D_025.zarr/2020


In [8]:
if 1:
    oc = occci.OCCCIDataset(**catalog['ftp']['occci'])
    soda = ocn.data.soda.SODADataset(**catalog['soda']['soda_3_15_2'])
    era5 = ocn.data.era5.ERA5Dataset(catalog['era5']['gcs_zarr'])
    cmems = ocn.data.cmems.CMEMSCatalog(catalog['cmems'])

2025-05-28 12:33:56.905 | INFO     | oceansoda_ethzv2.data.cmems:authenticate:113 - Successfully authenticated with CMEMS using provided credentials, and persisted credentials.


In [16]:
oc.write_timestep_to_disk(2014, 0)

2025-05-28 12:36:53.879 | DEBUG    | oceansoda_ethzv2.data.utils.checker:fix_timestep:244 - Time coordinate 2014-01-05 00:00:00 matches expected time 2014-01-05 00:00:00.
2025-05-28 12:36:53.879 | WARNING  | oceansoda_ethzv2.data.utils.checker:add_time_bnds:70 - Time bounds are missing, adding them.
2025-05-28 12:36:53.884 | INFO     | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:321 - Starting to write ../data/8D_025/chl_occci-8D_025.zarr for time 2014-01-05 00:00:00
2025-05-28 12:36:54.276 | SUCCESS  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:329 - Saved to: ../data/8D_025/chl_occci-8D_025.zarr/2014:2014-01-05 00:00
2025-05-28 12:36:54.278 | INFO     | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:321 - Starting to write ../data/8D_025/chl_occci_sigma_uncert-8D_025.zarr for time 2014-01-05 00:00:00
2025-05-28 12:36:54.604 | SUCCESS  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:329 - Saved to: ../data/8D_025/chl_occci_sigma_uncert-8D_025.zarr/2014:2014

In [15]:
era5.write_timestep_to_disk(2004, 0)

2025-05-28 12:35:04.341 | WARNING  | oceansoda_ethzv2.data.utils.checker:fix_timestep:240 - Time fixed. Time coordinate 2004-01-04 23:30:00 did not match expected time 2004-01-05 00:00:00. Adjusted to 2004-01-05 00:00:00.
2025-05-28 12:35:04.342 | WARNING  | oceansoda_ethzv2.data.utils.checker:add_time_bnds:70 - Time bounds are missing, adding them.
2025-05-28 12:35:04.349 | INFO     | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:321 - Starting to write ../data/8D_025/msl_era5-8D_025.zarr for time 2004-01-05 00:00:00
2025-05-28 12:35:11.241 | SUCCESS  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:329 - Saved to: ../data/8D_025/msl_era5-8D_025.zarr/2004:2004-01-05 00:00
2025-05-28 12:35:11.243 | INFO     | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:321 - Starting to write ../data/8D_025/u10_era5-8D_025.zarr for time 2004-01-05 00:00:00
2025-05-28 12:35:20.454 | SUCCESS  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:329 - Saved to: ../data/8D_025/u10_era5-

In [14]:
soda.write_timestep_to_disk(2002, 0)

2025-05-28 12:34:48.083 | WARNING  | oceansoda_ethzv2.data.utils.core:is_timestep_write_safe:368 - Given timestep [year=2002, index=0] already exists in ../data/8D_025/mld_soda-8D_025.zarr/2002


In [11]:
if 1:
    duacs = cmems.datasets['duacs']
    duacs.write_timestep_to_disk(2020, 0)

2025-05-28 12:34:37.730 | WARNING  | oceansoda_ethzv2.data.utils.checker:fix_timestep:240 - Time fixed. Time coordinate 2020-01-04 12:00:00 did not match expected time 2020-01-05 00:00:00. Adjusted to 2020-01-05 00:00:00.
2025-05-28 12:34:37.730 | WARNING  | oceansoda_ethzv2.data.utils.checker:add_time_bnds:70 - Time bounds are missing, adding them.
2025-05-28 12:34:37.739 | WARNING  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:311 - Skipping saving: Given timestep [year=2020, index=0] already exists in ../data/8D_025/adt_duacs-8D_025.zarr/2020
2025-05-28 12:34:37.743 | WARNING  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:311 - Skipping saving: Given timestep [year=2020, index=0] already exists in ../data/8D_025/sla_duacs-8D_025.zarr/2020
2025-05-28 12:34:37.749 | WARNING  | oceansoda_ethzv2.data.utils.zarr_utils:save_to_zarr:311 - Skipping saving: Given timestep [year=2020, index=0] already exists in ../data/8D_025/sla_duacs_sigma_uncert-8D_025.zarr/2020
2025-05-28 

In [54]:
out

<xarray.Dataset> Size: 21MB
Dimensions:                 (time: 1, lat: 720, lon: 1440)
Coordinates:
  * time                    (time) datetime64[ns] 8B 2020-06-28T12:00:00
  * lat                     (lat) float64 6kB -89.88 -89.62 ... 89.62 89.88
  * lon                     (lon) float64 12kB -179.9 -179.6 ... 179.6 179.9
Data variables:
    adt_duacs               (time, lat, lon) float32 4MB nan nan nan ... nan nan
    sla_duacs               (time, lat, lon) float32 4MB nan nan ... 0.0005156
    sla_duacs_sigma_uncert  (time, lat, lon) float32 4MB nan nan ... 0.01349
    flag_duacs              (time) float32 4B 1.0
    adt_duacs_sigma_regrid  (time, lat, lon) float32 4MB nan nan nan ... nan nan
    sla_duacs_sigma_regrid  (time, lat, lon) float32 4MB nan nan ... 0.0005227
Attributes:
    institution:              CLS, CNES
    references:               http://marine.copernicus.eu
    Conventions:              CF-1.6
    comment:                  Sea Surface Height measured by Altimetry and de...
    source:                   Altimetry measurements
    history:                  2024-10-23 12:55:06Z: Creation
    contact:                  servicedesk.cmems@mercator-ocean.eu
    title:                    DT merged all satellites Global Ocean Gridded S...
    processing_renamed_vars:  {'latitude': 'lat', 'longitude': 'lon', 'adt': ...
    processing_coarsen_grid:  {'boundary': 'pad', 'time': 8, 'lat': 2, 'lon': 2}